In [20]:
import numpy as np
import pandas as pd

train_df = pd.read_csv("data/train_embs.csv", index_col='seq_id')
train_df


,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,pH,tm
seq_id,,,,,,,,,,,,,,,,,,,,,
0,0.058200,0.013725,0.06010,-0.008590,-0.025600,0.072400,-0.04100,-0.12006,0.013440,0.010544,...,0.000931,-0.069500,-0.016970,-0.010460,-0.031200,0.039060,0.03665,0.019560,7.0,75.7
1,-0.029130,-0.022870,0.02185,-0.035400,0.017910,0.056520,0.00997,-0.13460,0.056180,0.056900,...,0.015366,-0.012980,-0.075700,0.023300,-0.074000,0.011300,0.04560,0.031430,7.0,50.5
2,0.035520,0.011470,-0.01593,0.033840,-0.039800,0.060670,-0.03340,-0.02382,-0.009575,0.021450,...,-0.013916,0.000024,0.011055,-0.028560,-0.066960,-0.006252,0.00259,0.016110,7.0,40.5
3,0.042330,-0.000993,0.02522,-0.005270,-0.006237,0.062800,-0.02255,-0.08520,0.097500,0.042000,...,0.036320,0.015450,-0.012400,-0.074100,-0.086200,-0.027620,0.01518,0.087900,7.0,47.2
4,0.016860,0.067100,0.01580,0.023530,-0.002163,0.002775,-0.02696,-0.04453,0.082600,-0.003222,...,-0.000928,-0.020130,0.052500,0.013245,-0.003328,0.034850,0.01533,-0.013245,7.0,49.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31385,0.025400,0.078700,0.08540,-0.010124,-0.017940,0.065500,-0.04626,-0.03635,0.003952,0.004055,...,-0.029400,0.046260,0.004345,0.017760,0.023670,0.020520,-0.05133,-0.018770,7.0,51.8
31386,0.052340,0.065900,0.07080,-0.000024,0.016600,0.046420,-0.09906,-0.06192,0.031100,-0.001596,...,0.034240,-0.013600,0.033500,-0.020630,0.024860,-0.038480,0.00240,-0.003370,7.0,37.2
31387,-0.082030,-0.016620,-0.03036,-0.025480,-0.054140,0.101300,-0.09375,-0.05370,0.019420,-0.024610,...,-0.002083,-0.141000,0.030980,-0.057700,-0.013450,-0.111940,0.01714,0.070600,7.0,64.6


In [33]:
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor

xgb = XGBRegressor(n_jobs=-1)
scores = cross_validate(xgb, train_df.iloc[:, :-1], train_df.iloc[:, -1], scoring='neg_mean_squared_error', verbose=2, cv=3)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=  23.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.3s remaining:    0.0s


[CV] END .................................................... total time=  25.9s
[CV] END .................................................... total time=  24.8s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished


In [34]:
scores

{'fit_time': array([23.2237308 , 25.84739256, 24.71513748]),
 'score_time': array([0.07836533, 0.0829587 , 0.08374047]),
 'test_score': array([-52.67420105, -77.72459455, -62.35393659])}

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_df.iloc[:,:-1], train_df.tm)

In [36]:
xgb = XGBRegressor(n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_valid)

In [40]:
y_pred[:10], y_valid[:10]

(array([45.722446, 70.2834  , 59.67066 , 83.79121 , 61.264286, 45.60595 ,
        57.73677 , 52.85321 , 39.4357  , 47.34829 ], dtype=float32),
 seq_id
 23070    53.1
 2449     75.3
 23737    42.6
 28159    90.1
 9481     60.4
 507      43.8
 24678    42.8
 7932     51.4
 15006    42.0
 28       45.2
 Name: tm, dtype: float64)

In [39]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(np.arange(100), y_pred[:100])

KeyError: 'Rectangle:kwdoc'